Jenkins Configuration Notebook
==============================

This notebook should be used to update the Jenkins configuration for the Menpo project en-masse. It works by completing three template projects with the state that changes between projects - namely, the name of the project (and hence the github URL) and the Python versions that need to be built.

Dependencies
------------

- java
- an internet connection (no SSH tunnel etc required!)

In [ ]:
# set your jenkins credentials here - and do not commit them!!
USERNAME = 'yourusername'
PASSWORD = 'yourpassword'

In [ ]:
def copy_and_yield(fsrc, fdst, length=1024*1024):
    """copy data from file-like object fsrc to file-like object fdst"""
    while 1:
        buf = fsrc.read(length)
        if not buf:
            break
        fdst.write(buf)
        yield
    
def download_file(url, dest_path):
    try:
        from urllib2 import urlopen  # Py2
    except ImportError:
        from urllib.request import urlopen  # Py3
    req = urlopen(url)
    with open(str(dest_path), 'wb') as fp:
        for _ in copy_and_yield(req, fp):
            pass
    req.close()

# downlad the CLI file for use
download_file('http://jenkins.menpo.org/jnlpJars/jenkins-cli.jar', 'jenkins-cli.jar')

In [ ]:
from subprocess import check_output, Popen, PIPE, STDOUT, CalledProcessError
from functools import partial
from collections import namedtuple

Project = namedtuple('Project', ['name', 'versions'])

T_OFF = '<disabled>true</disabled>'
T_ON = '<disabled>false</disabled>'
T_VERSIONS = '<string>@PYTHON_VERSIONS@</string>\n'
T_NAME = '@PROJECT_NAME@'

PROJECTS = [Project(*x) for x in
            [('menpo', (2, 34, 35)),
             ('menpofit', (2, 34, 35)),
             ('menpodetect', (2, 34, 35)),
             ('menpo3d', (2,)),
             ('menpobench', (2,)),
             ('menpocli', (2, 34, 35)),
             ('landmarkerio-server', (2,)),
             ('cyassimp', (2, 34, 35)),
             ('cyrasterize', (2, 34, 35)),
             ('cyvlfeat', (2, 34, 35)),
             ('cyffld2', (2, 34, 35)),
             ('cypico', (2, 34, 35)),
             ('conda-dlib', (2, 34, 35)),
             ('conda-boost', (2, 34, 35)),
             ('conda-joblib', (2, 34, 35)),
             ('conda-enum',  (2,)),
             ('conda-flann', (2,)),
             ('conda-cherrypy', (2, 34, 35)),
             ('conda-pathlib', (2,)),
             ('conda-eigen', (2,)),
             ('vrml97', (2,))]]

VERSION_TO_STR = {k: '<string>{}</string>\n        '.format(v) for k, v in 
                  {2: '2.7', 34: '3.4', 35: '3.5'}.items()}

JENKINS_CMD = ['java', '-jar', 'jenkins-cli.jar', '-s', 'http://jenkins.menpo.org']
AUTH = [
    '--username', USERNAME,
    '--password', PASSWORD
]

cmd_for_args = lambda args: JENKINS_CMD + list(args) + AUTH

    
def jcall(*args):
    try:
        return check_output(cmd_for_args(args)).decode()
    except CalledProcessError as e:
        print(e.returncode)
        print(e.output)


def jinput(stdin, cmd):
    p = Popen(cmd_for_args(cmd), stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    try:
        return p.communicate(input=stdin.encode())[0].decode()
    except CalledProcessError as e:
        print(e.returncode)
        print(e.output)

def fill_template(t, name, versions):
    return t.replace(T_NAME, name).replace(T_VERSIONS, versions).replace(T_OFF, T_ON)


get_job = partial(jcall, 'get-job')
create_job = lambda name, job: jinput(job, ['create-job', name])
update_job = lambda name, job: jinput(job, ['update-job', name])

In [ ]:
# Check we are successfully logged in:
print(jcall('who-am-i'))

In [ ]:
print('acquiring templates...')
T = get_job('TEMPLATE')
T_PR = get_job('TEMPLATE-pr')
T_TAG = get_job('TEMPLATE-tag')
SUFFIX_TO_TEMPLATE = {'': T, '-pr': T_PR, '-tag': T_TAG}
print('done.')
print('acquiring current jobs...')
JOBS = set(x for x in jcall('list-jobs').split('\n') if not x == '')
print('done.')

In [ ]:
for p in PROJECTS:
    versions = ' '.join([VERSION_TO_STR[v] for v in p.versions])
    for suffix, template in SUFFIX_TO_TEMPLATE.items():
        job = fill_template(template, p.name, versions)
        job_name = p.name + suffix
        if job_name in JOBS:
            print('{} already exists, updating.'.format(job_name))
            update_job(job_name, job)
        else:
            print('creating job: {}'.format(job_name))
            create_job(job_name, job)